In [ ]:
import numpy as np
import mesa
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from scipy.stats import zscore

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

## Social information

In [ ]:
# make the binary feature matrix
grid_size = 100
margin = 20
x_plot = np.meshgrid(range(grid_size+margin), range(grid_size+margin))
print(np.array(x_plot).shape)
s = np.zeros((grid_size + margin, grid_size + margin))

# set seed
np.random.seed(2)

# add some random 1s
n_other_agents = 5
for i in range(n_other_agents):
    ai = np.random.randint(margin//2, grid_size+margin//2)
    aj = np.random.randint(margin//2, grid_size+margin//2)
    print((ai, aj))
    s[ai, aj] = 1


In [ ]:
def plot_belief_style(_ax):
    # add border
    for _, spine in _ax.spines.items():
        spine.set_visible(True)
        spine.set_color('black')
        spine.set_linewidth(1)
    # ax.xaxis.set_ticks_position('top')
    # remove ticks
    _ax.set_xticks([])
    _ax.set_yticks([])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
cmap = 'binary'
sns.heatmap(s, ax=ax, cmap=cmap, cbar=False, square=True, vmin=0, vmax=1)
plot_belief_style(ax)
plt.show()


In [ ]:
x = np.meshgrid(range(grid_size+margin), range(grid_size+margin))
# X = np.array(X).reshape(2, -1).T
# y = s.flatten()

# select only the 1s
X = np.array(x).reshape(2, -1).T[s.flatten() == 1]
y = np.ones(X.shape[0])
print(X.shape, y.shape)

# add regular grid of every tenth mesh point with its value if not already in X
X = np.vstack([X, np.array(x)[:, ::20, ::20].reshape(2, -1).T])
y = np.hstack([y, np.zeros(X.shape[0] - y.shape[0])])
print(X.shape, y.shape)

# to reconstruct s
# s_reconstructed = y.reshape(50, 50)

In [ ]:
# plot selected points
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
cmap = 'binary'
sns.heatmap(s, ax=ax, cmap=cmap, cbar=False, square=True, vmin=0, vmax=1)
plot_belief_style(ax)
ax.scatter(X[:, 0], X[:, 1], color='red', s=10, alpha=0.5)
plt.show()


In [ ]:
gpc = GaussianProcessClassifier(kernel=RBF(12), random_state=0, optimizer=None)

In [ ]:
%%time

gpc.fit(X, y)

In [ ]:
%%time
x_plot = np.meshgrid(range(margin//2, grid_size + margin//2), range(margin//2, grid_size + margin//2))
s_prob = gpc.predict_proba(np.array(x_plot).reshape(2, -1).T)

In [ ]:
from ice_fishing_abm_gp.belief import generate_belief_matrix, construct_dataset_info

In [ ]:
# %prun gpc.fit(X, y)
# s_prob = gpc.predict_proba(X)

In [ ]:
agent_locs = np.array(np.meshgrid(range(grid_size+margin), range(grid_size+margin))).reshape(2, -1).T[s.flatten() == 1] - margin // 2

In [ ]:
%%time
X_new, y_new = construct_dataset_info(grid_size, margin, agent_locs, step_size=20)

In [ ]:
gpc = GaussianProcessClassifier(kernel=RBF(10), random_state=0, optimizer=None)

In [ ]:
%%time
# s_prob_plot = s_prob[:, 1].reshape(grid_size, grid_size)
s_prob_plot = generate_belief_matrix(grid_size, margin, X_new, y_new, gpc)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
cmap = 'viridis'

sns.heatmap(s_prob_plot, ax=ax, cmap=cmap, cbar=False, square=True)
# plot_belief_style(ax)
ax.scatter(agent_locs[:, 0], agent_locs[:, 1], color='red', s=10, alpha=0.5, marker='x')
ax.scatter(X_new[y_new==0, 0] - margin // 2, X_new[y_new==0, 1] - margin // 2, color='white', s=10, alpha=0.5, marker='o')
plt.show()

In [ ]:
agent_locs

## Individual features: success and loss
For these features, we want to use only the actual observations instead of the regular grid.

In [ ]:
success_locs = np.array([[50, 50]])
success_gpc = GaussianProcessClassifier(kernel=RBF(5), random_state=0, optimizer=None)

In [ ]:
X_success = np.vstack([success_locs, np.array([[20, 20]])]) + margin // 2
y_success = np.array([1, 0])
catch_prob_plot = generate_belief_matrix(grid_size, margin, X_success, y_success, success_gpc)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
cmap = 'viridis'

sns.heatmap(catch_prob_plot, ax=ax, cmap=cmap, cbar=False, square=True)
# plot_belief_style(ax)
ax.scatter(success_locs[:, 0], success_locs[:, 1], color='red', s=10, alpha=0.5, marker='x')
plt.show()

# GaussianProcessRegressor

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from ice_fishing_abm_gp.belief import generate_belief_mean_matrix

success_locs = np.array([[50, 50]])
gpr = GaussianProcessRegressor(kernel=RBF(5), random_state=0, optimizer=None)
gpr.fit(success_locs, [1])
success_m, success_std = generate_belief_mean_matrix(grid_size, gpr, return_std=True)  # success_cov


In [ ]:
# success_locs = np.array([[]])
# gpr = GaussianProcessRegressor(kernel=RBF(5), random_state=0, optimizer=None).fit(success_locs, [])

In [ ]:
social_locs = np.array([[20, 20], [60, 60]])
gpr = GaussianProcessRegressor(kernel=RBF(12), random_state=0, optimizer=None)
gpr.fit(social_locs, [1, 1])
social_m, social_std = generate_belief_mean_matrix(grid_size, gpr, return_std=True)  # social_cov

In [ ]:
failure_locs = np.array([[50, 30]])
gpr = GaussianProcessRegressor(kernel=RBF(5), random_state=0, optimizer=None)
gpr.fit(failure_locs, [1])
failure_m, failure_std = generate_belief_mean_matrix(grid_size, gpr, return_std=True)  # success_cov

In [ ]:
# combine two gaussian processes as a weighted sum of the two GPs
# belief_m = 0.5 * success_m.reshape(grid_size * grid_size) + 0.5 * social_m.reshape(grid_size * grid_size)
# belief_cov = success_cov + social_cov
ws, wc, wl = 1/3, 1/3, 1/3
belief_m = ws * success_m + wc * social_m + wl * (1 - failure_m)
belief_std = np.sqrt(wc**2 * success_std **2 + ws**2 * social_std**2 + wl**2 * failure_std**2)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(14, 5))
cmap = 'viridis'

sns.heatmap(success_m, ax=axes[0], cmap=cmap, cbar=True, square=True)
axes[0].set_title('Success GP mean')
sns.heatmap(success_std, ax=axes[1], cmap='Blues', cbar=True, square=True)
axes[1].set_title('Success GP std')

for ax in axes:
    plot_belief_style(ax)
    ax.scatter(success_locs[:, 0], success_locs[:, 1], color='red', s=10, alpha=0.5, marker='x')
plt.show()


In [ ]:
_, axes = plt.subplots(1, 2, figsize=(14, 5))
cmap = 'viridis'

sns.heatmap(1 - failure_m, ax=axes[0], cmap=cmap, cbar=True, square=True)
axes[0].set_title('Loss GP mean')
sns.heatmap(failure_std, ax=axes[1], cmap='Blues', cbar=True, square=True)
axes[1].set_title('Loss GP std')

for ax in axes:
    plot_belief_style(ax)
    ax.scatter(failure_locs[:, 0], failure_locs[:, 1], color='red', s=10, alpha=0.5, marker='x')
plt.show()

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(14, 5))
cmap = 'viridis'

sns.heatmap(social_m, ax=axes[0], cmap=cmap, cbar=True, square=True)
axes[0].set_title('Social GP mean')
sns.heatmap(social_std, ax=axes[1], cmap='Blues', cbar=True, square=True)
axes[1].set_title('Social GP std')

for ax in axes:
    plot_belief_style(ax)
    ax.scatter(social_locs[:, 0], social_locs[:, 1], color='red', s=10, alpha=0.5, marker='x')
plt.show()


In [ ]:
_, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()
cmap = 'viridis'
beta = 0.2
tau = 0.05

sns.heatmap(belief_m, ax=axes[0], cmap=cmap, cbar=True, square=True)
axes[0].set_title('Belief mean')
sns.heatmap(belief_std, ax=axes[1], cmap='Blues', cbar=True, square=True)
axes[1].set_title('Belief std')
belief_ucb = belief_m + beta * belief_std
sns.heatmap(belief_ucb, ax=axes[2], cmap=cmap, cbar=True, square=True)
axes[2].set_title(f'UCB (beta = {beta})')
belief_softmax = np.exp(belief_ucb / tau) / np.sum(np.exp(belief_ucb / tau))
sns.heatmap(belief_softmax, ax=axes[3], cmap='Reds', cbar=True, square=True)
axes[3].set_title(f'Softmax (tau = {tau})')

for ax in axes:
    plot_belief_style(ax)
    # ax.scatter(social_locs[:, 0], social_locs[:, 1], color='red', s=10, alpha=0.5, marker='x')
plt.tight_layout()
plt.show()

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(5, 5))
cmap = 'viridis'
beta = 0.2
sns.heatmap(belief_m + beta * belief_std, ax=ax, cmap=cmap, cbar=True, square=True)
plot_belief_style(ax)
plt.show()

In [ ]:
np.array(np.meshgrid(range(20), range(20))).reshape(2, -1).T

# GPJax

In [ ]:
# Enable Float64 for more stable matrix inversions.
from jax import config

config.update("jax_enable_x64", True)

from time import time
import blackjax
import jax
import jax.numpy as jnp
import jax.random as jr
import jax.scipy as jsp
import jax.tree_util as jtu
from jaxtyping import (
    Array,
    Float,
    install_import_hook,
)
import matplotlib.pyplot as plt
import optax as ox
import tensorflow_probability.substrates.jax as tfp
from tqdm import trange

with install_import_hook("gpjax", "beartype.beartype"):
    import gpjax as gpx

tfd = tfp.distributions
identity_matrix = jnp.eye
key = jr.key(123)
plt.style.use(
    "https://raw.githubusercontent.com/JaxGaussianProcesses/GPJax/main/docs/examples/gpjax.mplstyle"
)


import cola
from gpjax.lower_cholesky import lower_cholesky

In [ ]:
D = gpx.Dataset(X=X, y=y)
kernel = gpx.kernels.RBF()
meanf = gpx.mean_functions.Constant()
prior = gpx.gps.Prior(mean_function=meanf, kernel=kernel)
likelihood = gpx.likelihoods.Bernoulli(num_datapoints=grid_size**2)

In [ ]:
posterior = prior * likelihood
print(type(posterior))

In [ ]:
negative_lpd = jax.jit(gpx.objectives.LogPosteriorDensity(negative=True))

optimiser = ox.adam(learning_rate=0.01)

opt_posterior, history = gpx.fit(
    model=posterior,
    objective=negative_lpd,
    train_data=y,
    optim=ox.adamw(learning_rate=0.01),
    num_iters=1000,
    key=key,
)

In [ ]:




def construct_laplace(test_inputs: Float[Array, "N D"]) -> tfd.MultivariateNormalTriL:
    map_latent_dist = opt_posterior.predict(xtest, train_data=D)

    Kxt = opt_posterior.prior.kernel.cross_covariance(x, test_inputs)
    Kxx = opt_posterior.prior.kernel.gram(x)
    Kxx += identity_matrix(D.n) * jitter
    Kxx = cola.PSD(Kxx)

    # Kxx⁻¹ Kxt
    Kxx_inv_Kxt = cola.solve(Kxx, Kxt)

    # Ktx Kxx⁻¹[ H⁻¹ ] Kxx⁻¹ Kxt
    laplace_cov_term = jnp.matmul(jnp.matmul(Kxx_inv_Kxt.T, H_inv), Kxx_inv_Kxt)

    mean = map_latent_dist.mean()
    covariance = map_latent_dist.covariance() + laplace_cov_term
    L = jnp.linalg.cholesky(covariance)
    return tfd.MultivariateNormalTriL(jnp.atleast_1d(mean.squeeze()), L)